In [13]:
import numpy as np
import pandas as pd
import math

In [14]:
# parameter values
mu = 0.08
r = 0.04
sigma = 0.16
q = 0.02      #discount, hazard rate
K = 0.04      #fixed rate
sigma2 = sigma*sigma
num_years = 5
recovery = 0.4
notional = 100 #notional 100m
# coupon values
###############################################################################
# statistical measure(real world)      
EpR_i = mu - 0.5 * sigma2 - K
Rp = EpR_i * np.ones(num_years)                        

# risk-neutral measure
EqR_i = r - 0.5 * sigma2 - K
Rq = EqR_i * np.ones(num_years)                          

# swap values
###############################################################################
# discount curve
Z = np.exp(-r * np.arange(0, num_years + 1)) #discount factor
Z_month = np.exp(-(r / 12) * np.arange(0, num_years * 12 + 1))
PD = np.exp(-q * np.arange(0, num_years + 1))         #probability of default 
PD_month = np.exp(-(q / 12) * np.arange(0, num_years * 12 + 1))

# value (the cash flows)   #r
discounted_cf = Z[1:] * Rq    #discount factor * each month value    
# values discounted to today                              
V = np.cumsum(discounted_cf[::-1])[::-1]         
V_0 = V[0]

# value of swap at each point in time [V(0), V(1), ..., V(num_years-1)]
V_t = V / Z[:-1]                                            

# expected exposures
###############################################################################
# expected exposure at end of period
EE_e = EpR_i + V_t[1:]  # mu
EE_s = V_t

In [15]:
# r: risk neutral measure.  swap value, CVA, DVA
pos_v = []
neg_v = []
for i in range(5):
    temp_pos_v = []
    temp_neg_v = []
    for j in range(2**17):
        x = np.random.normal(0, 1)
        coupon =  (r - 0.5 * sigma2) + math.sqrt(5) * 0.2 * sigma * x
        if coupon >= K:
            temp_pos_v.append(coupon - K)
        else:
            temp_neg_v.append(coupon - K)
    pos_v.append(np.mean(temp_pos_v))
    neg_v.append(np.mean(temp_neg_v))

In [16]:
# mu: real world measure. exposure
pos_e = []
neg_e = []
for i in range(5):
    temp_pos_e = []
    temp_neg_e = []
    for j in range(2**17):
        x = np.random.normal(0, 1)
        coupon =  (mu - 0.5 * sigma2) + math.sqrt(5) * 0.2 * sigma * x
        if coupon >= K:
            temp_pos_e.append(coupon - K)
        else:
            temp_neg_e.append(coupon - K)
    pos_e.append(np.mean(temp_pos_e))
    neg_e.append(np.mean(temp_neg_e))

In [17]:
#value   # r: risk neutral measure.  swap value, CVA, DVA
discounted_pos_v = Z[1:] * pos_v
discounted_neg_v = Z[1:] * neg_v
#values discounted to today
V_pos = np.cumsum(discounted_pos_v[::-1])[::-1]
V_neg = np.cumsum(discounted_neg_v[::-1])[::-1]
#value of swap at each point in time [V(0), V(1), ..., V(num_years-1)]
V_pos_t = V_pos / Z[:-1]
V_pos_t = np.append(V_pos_t, 0)
V_neg_t = V_neg / Z[:-1]
V_neg_t = np.append(V_neg_t, 0)

In [18]:
## value of swap at time zero
V_0 = V_pos_t[0] * notional
V_0

23.36746305606918

In [19]:
#value  # mu: real world measure. exposure
discounted_pos_e = Z[1:] * pos_e
discounted_neg_e = Z[1:] * neg_e
#values discounted to today
E_pos = np.cumsum(discounted_pos_e[::-1])[::-1]
E_pos_month = E_pos.repeat(12)
E_neg = np.cumsum(discounted_neg_e[::-1])[::-1]
E_neg_month = E_neg.repeat(12)
#value of swap at each point in time [V(0), V(1), ..., V(num_years-1)] by month
E_pos_t = E_pos / Z[:-1]
E_pos_t_month = E_pos_month / Z_month[:-1]
E_pos_t_month = np.append(E_pos_t_month, 0)

E_neg_t = E_neg / Z[:-1]
E_neg_t_month = E_neg_month / Z_month[:-1]
E_neg_t_month = np.append(E_neg_t_month, 0)

In [20]:
#EPE
EPE = np.average(E_pos_t_month[1:]) * notional
#ENE
ENE = np.average(E_neg_t_month[1:]) * notional
#EE
EE = (np.sum(E_pos_t_month[1:]) + np.sum(E_neg_t_month[1:])) / (num_years * 12 *2) * notional

EE,EPE,ENE

(2.70838150084662, 18.50462210707294, -13.087859105379698)

In [21]:
## potential future exposure with 95 percentile
PFE = np.percentile(E_pos_t_month[1:], 95) * notional
PFE

31.11410674987588

In [22]:
#by month  #value   # r: risk neutral measure.  swap value, CVA, DVA
discounted_pos_v = Z[1:] * pos_v
discounted_neg_v = Z[1:] * neg_v
#values discounted to today
V_pos = np.cumsum(discounted_pos_v[::-1])[::-1]
V_pos_month = V_pos.repeat(12)
V_neg = np.cumsum(discounted_neg_v[::-1])[::-1]
V_neg_month = V_neg.repeat(12)
#value of swap at each point in time [V(0), V(1), ..., V(num_years-1)] by month
V_pos_t = V_pos / Z[:-1]
V_pos_t_month = V_pos_month / Z_month[:-1]
V_pos_t_month = np.append(V_pos_t_month, 0)

V_neg_t = V_neg / Z[:-1]
V_neg_t_month = V_neg_month / Z_month[:-1]
V_neg_t_month = np.append(V_neg_t_month, 0)

In [23]:
#CVA
CVA = np.sum((1 - recovery) * PD_month[:-1] * (1-np.exp(-q / 12)) * V_pos_t_month[1:] * Z_month[1:] * notional)
CVA

0.7680983836886364

In [24]:
#DVA
DVA = np.sum((1 - recovery) * PD_month[:-1] * (1-np.exp(-q / 12)) * V_neg_t_month[1:] * Z_month[1:] * notional)
DVA

-0.9058288376319021